In [1]:
import pyarrow.parquet as pq
import s3fs
import os
import pyarrow.compute as pc

In [2]:
%env AWS_ACCESS_KEY_ID=projet-ape-sa
%env AWS_SECRET_ACCESS_KEY=0obEe7LB59g1Zj65nueDa84OQvrlyfPH

env: AWS_ACCESS_KEY_ID=projet-ape-sa
env: AWS_SECRET_ACCESS_KEY=0obEe7LB59g1Zj65nueDa84OQvrlyfPH


In [3]:
del os.environ["AWS_SESSION_TOKEN"]

In [4]:
def get_filesystem():
    fs = s3fs.S3FileSystem(
        client_kwargs={"endpoint_url": "https://" + "minio.lab.sspcloud.fr"},
        key=os.getenv("AWS_ACCESS_KEY_ID"),
        secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
    )
    return fs

In [5]:
fs = get_filesystem()

In [6]:
test1 = (
    pq.read_table('projet-ape/extractions/sirene db/base_sirene_ape.parquet', filesystem=fs)
        #.filter(pc.not_equal(pc.field("activ_pr_lib"), pc.field("activ_pr_lib_et")))
        .to_pandas()
)


In [7]:
test1

,liasse_numero,id,niveau_auto_id,evenement_type,cj,domas,ssdom,liasse_type,activ_pr_lib,activ_pr_lib_et,activ_nat_et,activ_surf_et,code_retour_apet,libelle_activite_apet,nature_activites_apet,surface_apet,date_modification,mode_calcul_ape,apet_finale
0,J00012803128,51107,NaN,01P,None,None,None,C,Commerce et artisanat de détail Vente à distance,Commerce et artisanat de détail Vente à distance,99,NaN,CCF,Commerce et artisanat de détail Vente à distance,99,NaN,2023-01-02 09:54:17.128654+00:00,FASTTEXT,4791A
1,J00012825980,52023,NaN,01P,None,None,None,X,accompagnement du deuil- coaching,accompagnement du deuil- coaching,None,NaN,CCF,accompagnement du deuil- coaching,None,NaN,2023-01-02 10:12:53.806655+00:00,FASTTEXT,9609Z
2,J00012827192,52079,NaN,01P,None,None,None,C,Ventes d'articles diverses et varier tout ce q...,Ventes d'articles diverses et varier tout ce q...,99,NaN,CCF,Ventes d'articles diverses et varier tout ce q...,99,NaN,2023-01-02 10:17:46.646212+00:00,FASTTEXT,4719B
3,J00012828810,52157,NaN,01P,None,None,None,C,"Elaboration de projets de médiation numérique,...","Elaboration de projets de médiation numérique,...",99,NaN,C_C,"Elaboration de projets de médiation numérique,...",99,NaN,2023-01-03 15:52:49.292925+00:00,NAF,6202A
4,J00012846200,52604,NaN,01P,None,None,None,M,Création de bouquets artisanal,Création de bouquets artisanal,04,NaN,C_C,Création de bouquets artisanal,04,NaN,2023-01-02 11:54:39.490479+00:00,NAF,4776Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156214,J00053493375,2696366,2.0,01P,None,None,None,C,vente à distance sur catalogue spécialisé 479...,vente à distance sur catalogue spécialisé 479...,99,NaN,CCF,vente à distance sur catalogue spécialisé 479...,99,NaN,2024-01-24 22:11:25.502911+00:00,AUTO,4791B
1156215,J00053493334,2696367,2.0,01P,None,None,None,C,Livreur à domicile via Uber eats et/ou Deliveroo,Livreur à domicile via Uber eats et/ou Deliveroo,99,NaN,CCF,Livreur à domicile via Uber eats et/ou Deliveroo,99,NaN,2024-01-24 22:11:30.280291+00:00,AUTO,5320Z
1156216,J00053493409,2696369,2.0,01P,None,None,None,M,"Activité d'entretien de locaux (ménage, repurg...","Activité d'entretien de locaux (ménage, repurg...",99,NaN,CCF,"Activité d'entretien de locaux (ménage, repurg...",99,NaN,2024-01-24 22:12:14.534373+00:00,AUTO,8121Z
1156217,J00052724390,2696372,2.0,01P,None,None,None,X,Homme toutes mains - services à la personne pe...,Homme toutes mains - services à la personne pe...,None,NaN,CCF,Homme toutes mains - services à la personne pe...,None,NaN,2024-01-24 22:12:29.763259+00:00,AUTO,8121Z


In [14]:
test2 = (
    pq.read_table('projet-ape/extractions/sirene db/base_sirene_ape.parquet', filesystem=fs)
        .filter(pc.not_equal(pc.field("activ_pr_lib"), pc.field("libelle_activite_apet")))
        .to_pandas()
)


In [32]:
test3 = (
    pq.read_table('projet-ape/extractions/sirene db/base_sirene_ape.parquet', filesystem=fs)
        .select(["liasse_numero", "id",	"evenement_type", "liasse_type", "activ_nat_et", "activ_surf_et",  "activ_pr_lib_et", "libelle_activite_apet", "nature_activites_apet", "surface_apet", "date_modification", "mode_calcul_ape", "apet_finale"])
        .filter(pc.not_equal(pc.field("activ_nat_et"), pc.field("nature_activites_apet")))
        .to_pandas()
)
test3.shape

(103, 13)

In [33]:
test3.head()

,liasse_numero,id,evenement_type,liasse_type,activ_nat_et,activ_surf_et,activ_pr_lib_et,libelle_activite_apet,nature_activites_apet,surface_apet,date_modification,mode_calcul_ape,apet_finale
0,J00012964797,57584,01P,M,99,NaN,Sous-traitance pour l'assemblage de matériel,Sous-traitance pour l'assemblage de matériel,05,NaN,2023-01-03 12:13:45.300734+00:00,FASTTEXT,3320B
1,J00013416557,75538,01M,C,99,NaN,Négociatrice des achats. Apporteur d'affaire. ...,Négociatrice des achats. Apporteur d'affaire. ...,12,NaN,2023-01-10 11:22:41.204158+00:00,FASTTEXT,4619B
2,J00013419445,75584,01M,M,99,NaN,"nettoyage, maintenance, réparation, installati...","nettoyage, maintenance, réparation, installati...",12,NaN,2023-01-10 11:48:05.908516+00:00,FASTTEXT,4322B
3,J00013430095,75958,01M,C,99,NaN,Le commerce et la fabrication de tous produits...,Le commerce et la fabrication de tous produits...,12,NaN,2023-01-10 12:32:00.502299+00:00,FASTTEXT,5610C
4,J00013590351,82059,01M,C,99,NaN,"Activité de holding, prise de participations o...","Activité de holding, prise de participations o...",12,NaN,2023-01-12 08:42:32.172783+00:00,FASTTEXT,6420Z
